# Install Dependendcies

In [ ]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 14.0 MB/s 
     |████████████████████████████████| 7.6 MB 53.2 MB/s 
     |████████████████████████████████| 163 kB 69.4 MB/s 


In [ ]:
!nvidia-smi

Sun Nov  6 11:24:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# from google.colab import drive 
# drive.mount('/content/gdrive')
# !ln -s /content/gdrive/My\ Drive/ /mydrive
# !ls /mydrive

# Download the Dataset

In [ ]:
!gdown --id 1-n9aJG1zdWHL0UjuRWwa8Rro-ogoWxN7
!unzip deception_dataset.zip && rm deception_dataset.zip && rm -r __MACOSX

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: deception_dataset/restaurant/deceptive_MTurk/85.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._85.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/153.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._153.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/147.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._147.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/146.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._146.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/152.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._152.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/84.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._84.txt  
  inflating: deception_dataset/restaurant/deceptive_MT

# Data Helpers

In [ ]:
import numpy as np
import string
import os
import io
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

SEQUENCE_LEN = 200


def read_from_dir(in_fold_dir):
    reviews = []
    for filename in os.listdir(in_fold_dir):
        if filename.endswith(".txt"):
            # read from file containing the text review
            in_filepath = in_fold_dir + filename
            with io.open(in_filepath, 'r', encoding='windows-1252') as f:
                review = f.read()
                # if len(review) > SEQUENCE_LEN:
                #     review = review[:200]
                reviews.append(review)    

    # print("number of reviews: {}, name: {}".format(len(reviews), in_fold_dir)) 
    print(reviews[:2])         
    return reviews


def load_data():
    trustworthy_hotel_positive = read_from_dir('deception_dataset/hotel/positive/truthful/') 
    trustworthy_hotel_negative = read_from_dir('deception_dataset/hotel/negative/truthful/')
    trustworthy_restaurant = read_from_dir('deception_dataset/restaurant/truthful/')
    trustworthy_doctor = read_from_dir('deception_dataset/doctor/truthful/')[:200]

    untrustworthy_hotel_positive = read_from_dir('deception_dataset/hotel/positive/deceptive_turker/')
    untrustworthy_hotel_negative = read_from_dir('deception_dataset/hotel/negative/deceptive_turker/') 
    untrustworthy_hotel_expert_positive =  read_from_dir('deception_dataset/hotel/positive/deceptive_expert/')
    untrustworthy_hotel_expert_negative =  read_from_dir('deception_dataset/hotel/negative/deceptive_expert/')
    
    untrustworthy_restaurant = read_from_dir('deception_dataset/restaurant/deceptive_MTurk/')[:200]
    untrustworthy_doctor = read_from_dir('deception_dataset/doctor/deceptive_MTurk/')

    TRUSTWORTHY_REVIEWS_DICT = {}
    UNTRUSTWORTHY_REVIEWS_DICT = {}

    TRUSTWORTHY_REVIEWS_DICT["cross_domain"] = {
        "hotel":trustworthy_hotel_positive + trustworthy_hotel_negative,
        "restaurant":trustworthy_restaurant,
        "doctor":trustworthy_doctor
    }

    UNTRUSTWORTHY_REVIEWS_DICT["cross_domain"] = { 
        "hotel":untrustworthy_hotel_positive + untrustworthy_hotel_negative,
        "restaurant":untrustworthy_restaurant,
        "doctor":untrustworthy_doctor[:200]
    }

    TRUSTWORTHY_REVIEWS_DICT["mixed_domain"] = {
        "hotel":trustworthy_hotel_positive + trustworthy_hotel_negative,
        "restaurant":trustworthy_restaurant,
        "doctor":trustworthy_doctor
    }

    UNTRUSTWORTHY_REVIEWS_DICT["mixed_domain"] = { 
        "hotel":untrustworthy_hotel_positive + untrustworthy_hotel_expert_positive + untrustworthy_hotel_negative + untrustworthy_hotel_expert_negative,
        "restaurant":untrustworthy_restaurant,
        "doctor":untrustworthy_doctor
    }

    return TRUSTWORTHY_REVIEWS_DICT, UNTRUSTWORTHY_REVIEWS_DICT


TRUSTWORTHY_REVIEWS_DICT, UNTRUSTWORTHY_REVIEWS_DICT = load_data()

['Well located and well staffed. The Amalfi was a clean and comfortable place to stay. It lacks a restaurant or bar but many are near by. Free continental breakfast and evening cocktail. \n', "Just putting in a good word for the Hard Rock Hotel in Chicago. The reviews I read before going were mixed so we were not sure what to expect. The location was perfect. The hotel was beautiful. Our view was awesome. The room was large enough and very clean. The bed was very comfy. The staff were helpful - especially the concierge, Jared. I couldn't find anything negative to say. If you want to stay in downtown Chicago - you can't go wrong with this cool hotel. \n"]
['We chose to stay at a small hotel because we did were looking for a hotel with less crowds. We anticipated we would get better service in a smaller hotel, than some of the other larger high rise hotels. We did not experience very good service at the Talbott Hotel. When we checked Irena seemed very robotic and aloof. The front desk cl

In [ ]:
def get_reviews_from_dict(reviews_dict, keys = []):
    if not keys:
        raise Exception("Invalid key: key must be one of the following: \'hotel\', \'restaurant\', \'doctor\'")

    reviews = []
    for key in keys:
        if key in reviews_dict:
            reviews += reviews_dict[key]
        else:
            raise Exception("Invalid key: key must be one of the following: \'hotel\', \'restaurant\', \'doctor\'")
    return reviews

def get_reviews(review_type="", keys = [], data_name="cross_domain"):
    if not review_type:
        raise Exception("Invalid review_type: review_type must be either \'trustworthy\' or \'untrustworthy\'.")
    if not keys:
        raise Exception("Invalid key: key must be one of the follwowing: \'hotel\', \'restaurant\', \'doctor\'")

    if review_type=="trustworthy":
        return get_reviews_from_dict(reviews_dict=TRUSTWORTHY_REVIEWS_DICT[data_name], keys=keys)
    elif review_type=="untrustworthy":
        return get_reviews_from_dict(reviews_dict=UNTRUSTWORTHY_REVIEWS_DICT[data_name], keys=keys)
    else:
        raise Exception("Invalid review_type: review_type must be either \'trustworthy\' or \'untrustworthy\'.")


def get_data(keys=[], data_name="cross_domain"):
    if data_name != "cross_domain" and data_name != "mixed_domain":
        raise Exception("Invalid dataset name")

    trustworthy_reviews   = get_reviews(review_type="trustworthy", keys=keys, data_name=data_name)
    untrustworthy_reviews = get_reviews(review_type="untrustworthy", keys=keys, data_name=data_name)
    
    reviews = trustworthy_reviews + untrustworthy_reviews

    # generate labels
    labels_trustworthy   = [[1,0] for _ in range(len(trustworthy_reviews   ))]
    labels_untrustworthy = [[0,1] for _ in range(len(untrustworthy_reviews ))]
    labels = labels_trustworthy + labels_untrustworthy

    print("dataset_name: {}".format(data_name+"_"+keys[0]))
    print("trustworthy_reviews: {}".format(len(trustworthy_reviews)))
    print("untrustworthy_reviews: {}".format(len(untrustworthy_reviews)))
    print("reviews len: ", len(reviews))
    print("labels len: ", len(labels))
    print()
        
    return reviews, labels

In [ ]:
import numpy as np
import os
import re
import itertools
import scipy.sparse as sp
from collections import Counter
import torch


SEQUENCE_LEN = 200


def build_vocab(sentences, vocab_size=50000):
    word_counts = Counter(itertools.chain(*sentences)) 
    vocabulary_inv = [x[0] for x in word_counts.most_common(vocab_size)]
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    # append <UNK/> symbol to the vocabulary
    vocabulary['<UNK/>'] = len(vocabulary)
    vocabulary_inv.append('<UNK/>')
    return [vocabulary, vocabulary_inv]


def build_input_data(sentences, vocabulary):
    x = np.array([[vocabulary[word] if word in vocabulary else vocabulary['<UNK/>'] for word in sentence] for sentence in sentences])
    return x


def load_data(fold = 1, vocab_size=30001):
    # Load data
    trustworthy_reviews_for_training, trustworthy_reviews_for_testing, untrustworthy_reviews_for_training, untrustworthy_reviews_for_testing = data.load_data(fold=fold)
    train = trustworthy_reviews_for_training + untrustworthy_reviews_for_training
    test  = trustworthy_reviews_for_testing + untrustworthy_reviews_for_testing

    # generate labels
    train_labels1 = [[1,0] for _ in range(len(trustworthy_reviews_for_training   ))]
    train_labels0 = [[0,1] for _ in range(len(untrustworthy_reviews_for_training ))]
    test_labels1  = [[1,0] for _ in range(len(trustworthy_reviews_for_testing    ))]
    test_labels0  = [[0,1] for _ in range(len(untrustworthy_reviews_for_testing  ))]

    train_labels = np.concatenate((np.array(train_labels1), np.array(train_labels0)), axis=0)
    test_labels  = np.concatenate((np.array(test_labels1),  np.array(test_labels0)),  axis=0)

    # convert word2idx
    vocabulary, vocabulary_inv = build_vocab(train + test, vocab_size=vocab_size)
    train = build_input_data(train, vocabulary)
    test  = build_input_data(test,   vocabulary)
    
    return train, train_labels, train_labels, train, test_labels, test_labels, vocabulary, vocabulary_inv


def batch_iter(data, batch_size, num_epochs):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int(len(data)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_data = data[shuffle_indices]
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [ ]:
def get_cross_domain_dataset(test_keys=[]):
    train_keys = ["hotel"]

    train_reviews, train_labels = get_data(keys=train_keys, data_name="cross_domain")
    test_reviews,  test_labels  = get_data(keys=test_keys, data_name="cross_domain")
    all_reviews = train_reviews + test_reviews
    
    return train_reviews, train_labels, test_reviews, test_labels

def get_cross_domain_dataset_restaurant():
    return get_cross_domain_dataset(test_keys=["restaurant"])

def get_cross_domain_dataset_doctor():
    return get_cross_domain_dataset(test_keys=["doctor"])

# Review Dataset

In [ ]:
import torch

class UntrustworthyReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels    = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_data, train_labels, test_data, test_labels = get_cross_domain_dataset_doctor()

dataset_name: cross_domain_hotel
trustworthy_reviews: 800
untrustworthy_reviews: 800
reviews len:  1600
labels len:  1600

dataset_name: cross_domain_doctor
trustworthy_reviews: 200
untrustworthy_reviews: 200
reviews len:  400
labels len:  400



# Hyperparameters

In [ ]:
import torch.nn as nn

BATCH_SIZE = 16 # 48
LR = 1e-3
EPOCHS = 15
cross_entropy = nn.NLLLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

target_names = ["trustworthy", "untrustworthy"]

# Define Transformer-based Architecture for Fine-tuning

In [ ]:
class TransformerArchitecture(nn.Module):

    def __init__(self, bert):
      
      super(TransformerArchitecture, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, input_ids, attention_mask):

      #pass the inputs to the model  
      x = self.bert(input_ids, attention_mask=attention_mask)
      # print(x[0])
      
      x = self.fc1(x[0])

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x



# Define Training Method

In [ ]:
# function to train the model
cross_entropy = nn.NLLLoss()

def train(model, train_dataloader, optimizer):
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds, total_labels = [], []
  
  # iterate over batches
  for step, batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    input_ids      = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels         = batch['labels'].to(device)

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(input_ids, attention_mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # ADDED

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds = preds.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)
    total_labels.append(labels)
    
  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of cla
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds   = np.concatenate(total_preds, axis=0)
  total_labels  = np.concatenate(total_labels, axis=0)


  # returns the loss and predictions
  return avg_loss, total_preds, total_labels

# Define Evaluating Method

In [ ]:
def evaluate(model, valid_dataloader):
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds, total_labels = [], []

  # iterate over batches
  for step, batch in enumerate(valid_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(valid_dataloader)))

    # push the batch to gpu
    input_ids      = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels         = batch['labels'].to(device)

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(input_ids, attention_mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds, labels)

      total_loss = total_loss + loss.item()
      # print('valid_loss ', loss.item())

      # model predictions are stored on GPU. So, push it to CPU
      preds = preds.detach().cpu().numpy()
      labels = labels.detach().cpu().numpy()

      # append the model predictions
      total_preds.append(preds)
      total_labels.append(labels)

      # total_preds.append(preds) #you are adding twice to the list

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(valid_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  total_labels  = np.concatenate(total_labels, axis=0)

  return avg_loss, total_preds, total_labels

# Train and Validate

In [ ]:
from sklearn.metrics import classification_report


def train_and_validate(model, model_name, train_dataloader, valid_dataloader, optimizer):
  # set initial loss to infinite
  best_valid_loss = float('inf')
  best_accuracy = 0

  # empty lists to store training and validation loss of each epoch
  train_losses=[]
  valid_losses=[]

  for epoch in range(EPOCHS):
      
      print('\n Epoch {:} / {:}'.format(epoch + 1, EPOCHS))
      
      # train model
      train_loss, train_preds, train_labels = train(model, train_dataloader, optimizer)
      
      # evaluate model
      valid_loss, valid_preds, valid_labels = evaluate(model, valid_dataloader)
      
      # append training and validation loss
      train_losses.append(train_loss)
      valid_losses.append(valid_loss)
      
      print(f'\nTraining Loss: {train_loss:.3f}')
      print(f'Validation Loss: {valid_loss:.3f}')
     
      train_labels = np.argmax(train_labels, axis = 1)
      train_preds = np.argmax(np.argmax(train_preds, axis = 1), axis=1)  # inner np.argmax try np.softmax
      train_report = classification_report(train_labels, train_preds, target_names=target_names, output_dict=True)
      add(train_report, report, stage='train', metrics=metrics, model_name=model_name)

      valid_labels = np.argmax(valid_labels, axis = 1)
      valid_preds = np.argmax(np.argmax(valid_preds, axis = 1), axis=1)
      valid_report = classification_report(valid_labels, valid_preds, target_names=target_names, output_dict=True)
      add(valid_report, report, stage='valid', metrics=metrics, model_name=model_name)


      # print the best results for a given model (and save its weights)
      current_accuracy = 0
      for metric in metrics:
        if metric == "accuracy":
          current_accuracy += valid_report[metric]
        else:
          current_accuracy += valid_report['macro avg'][metric]

      if best_accuracy < current_accuracy:
        best_accuracy = current_accuracy
        print('Best Accuracy for the model {:} at {:} epoch'.format(model_name, epoch + 1))
        print(valid_report)
        # torch.save(model.state_dict(), model_name + '-best.pt')

      # # save the best model
      # if valid_loss < best_valid_loss:
      #     best_valid_loss = valid_loss
      #     torch.save(model.state_dict(), 'best.pt')

  return report

In [ ]:
def freeze_parameters(transformer_model):
  # freeze all the parameters
  for param in transformer_model.parameters():
      param.requires_grad = False

In [ ]:
from transformers import AutoModel
from transformers import AutoTokenizer
from torch.optim import AdamW


def train_and_evaluate(model_name):
  transformer_model = AutoModel.from_pretrained(model_name)
  freeze_parameters(transformer_model)

  # Extend the model for fine-tuning
  model_architecture = TransformerArchitecture(transformer_model)
  model_architecture.to(device)

  # Define optimizer
  optimizer = AdamW(model_architecture.parameters(), lr = LR)

  tokenizer = AutoTokenizer.from_pretrained(model_name)                                            
  train_encodings  = tokenizer(train_data, truncation=True, padding=True)
  test_encodings   = tokenizer(test_data, truncation=True, padding=True)

  train_dataset    = UntrustworthyReviewDataset(train_encodings, train_labels)
  valid_dataset    = UntrustworthyReviewDataset(test_encodings, test_labels)

  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  valid_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

  report = train_and_validate(model_architecture, model_name, train_dataloader, valid_dataloader, optimizer)
  
  return report

# Plotting Methods

In [ ]:
from matplotlib import pyplot

def initialize_report(stages=['train', 'valid'], metrics=['precision', 'recall', 'f1-score', 'accuracy'], model_names=[]):
  report = {}
  # models = ["m1"], ["m2"]
  for model_name in model_names:
    report[model_name] = {}
    for stage in stages:
        report[model_name][stage] = {}
        for metric in metrics:
            report[model_name][stage][metric] = []

  return report

def add(cls_report, report, stage, metrics, model_name):
  metrics = ['precision', 'recall', 'f1-score', 'accuracy']
  for metric in metrics:
    if metric == "accuracy":
      report[model_name][stage][metric].append(cls_report[metric])
    else:
      report[model_name][stage][metric].append(cls_report['macro avg'][metric])
    
    
def plot_results(report, stages, metrics, model_names):
  for stage in stages:
    pyplot.subplot(212)
    for metric in metrics:
      pyplot.title(stage + ' ' + metric)
      for model_name in model_names:
        pyplot.xlabel('epochs')
        pyplot.ylabel(metric) 

        eval_metric = report[model_name][stage][metric]
        epochs = [(i+1) for i in range(len(eval_metric))]

        pyplot.plot(epochs, eval_metric, label=model_name, 
                    marker='o', linestyle='--')

      pyplot.legend()
      pyplot.show()

# Run the Experiment

In [20]:
stages = ['train', 'valid']
metrics = ['precision', 'recall', 'f1-score', 'accuracy']
model_names = ['roberta-base']


report = initialize_report(stages=['train', 'valid'], metrics=['precision', 'recall', 'f1-score', 'accuracy'], model_names=model_names)
print(report)


for model_name in model_names:
  train_and_evaluate(model_name)

{'roberta-base': {'train': {'precision': [], 'recall': [], 'f1-score': [], 'accuracy': []}, 'valid': {'precision': [], 'recall': [], 'f1-score': [], 'accuracy': []}}}


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 Epoch 1 / 15
  Batch    50  of    100.

Evaluating...
  Batch    50  of    100.

Training Loss: 2.730
Validation Loss: 1.201
Best Accuracy for the model roberta-base at 1 epoch
{'trustworthy': {'precision': 0.5436005625879043, 'recall': 0.96625, 'f1-score': 0.6957695769576958, 'support': 800}, 'untrustworthy': {'precision': 0.848314606741573, 'recall': 0.18875, 'f1-score': 0.30879345603271985, 'support': 800}, 'accuracy': 0.5775, 'macro avg': {'precision': 0.6959575846647387, 'recall': 0.5775, 'f1-score': 0.5022815164952078, 'support': 1600}, 'weighted avg': {'precision': 0.6959575846647387, 'recall': 0.5775, 'f1-score': 0.5022815164952078, 'support': 1600}}

 Epoch 2 / 15
  Batch    50  of    100.

Evaluating...
  Batch    50  of    100.

Training Loss: 1.859
Validation Loss: 0.818
Best Accuracy for the model roberta-base at 2 epoch
{'trustworthy': {'precision': 0.5612903225806452, 'recall': 0.97875, 'f1-score': 0.7134396355353075, 'support': 800}, 'untrustworthy': {'precision': 0.9

In [21]:
report

{'roberta-base': {'train': {'precision': [0.4828293755043871,
    0.47326649958228906,
    0.49326548307531715,
    0.5249475062888506,
    0.5428624098481836,
    0.5413933870495793,
    0.5683533020106089,
    0.5726644855786948,
    0.5859226112225241,
    0.5784313725490196,
    0.5940525307066289,
    0.5915216468590832,
    0.5620414643786931,
    0.5818929350562175,
    0.6011454392601934],
   'recall': [0.49374999999999997,
    0.485,
    0.495,
    0.520625,
    0.536875,
    0.53625,
    0.560625,
    0.5656249999999999,
    0.578125,
    0.5725,
    0.586875,
    0.58625,
    0.558125,
    0.57625,
    0.5962500000000001],
   'f1-score': [0.3980369315489959,
    0.42152604098108903,
    0.4602457284762129,
    0.4989202204391716,
    0.5201163766357875,
    0.5213821863223026,
    0.5478443616618006,
    0.5548436968949948,
    0.5683313488566176,
    0.5642618251313904,
    0.5788398902658094,
    0.5802049512987013,
    0.5510396694077113,
    0.5688222902341111,
    0.591